In [ ]:
# using NonlinearSolve, ArbNumerics, LinearSolve
# ArbNumerics.setprecision(256)

# function f!(F, x, p)
#     F[1] = x[1]^2 + x[2]^2 - ArbFloat(1)
#     F[2] = x[1] - x[2]
# end

# x0 = ArbFloat[ArbFloat(0.5), ArbFloat(0.5)]
# prob = NonlinearProblem(f!, x0)

# sol = solve(prob, NewtonRaphson(; autodiff = AutoFiniteDiff()),
#             linsolve = KrylovJL_GMRES(),
#             abstol = ArbFloat(1e-50), reltol = ArbFloat(1e-50))

SciMLBase.CommonKwargError: Unrecognized keyword arguments found.
The only allowed keyword arguments to `solve` are:
(:dense, :saveat, :save_idxs, :save_discretes, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :maxtime, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :progress_id, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :u0, :p, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise, :batch, :nlsolve_kwa

This notebook is a playground for understanding how to transform my code into arbitrary precision. We will use the ArbNumerics.jl package, along with Julia's native parametric types in order to achieve this. 

# Imports

In [1]:
using ArbNumerics
using Plots
using LinearAlgebra
using SpecialFunctions

# **Basics of using ArbNumerics**

## The `ArbFloat` and `ArbReal` types

In [18]:
setprecision(ArbFloat, 113); setextrabits(24);

In [30]:
# create an ArbFloat and evaluate a Bessel function
x = ArbReal("100.0")
nu = ArbReal("1.0")
y = besselk(nu, x)

println(y)
println(typeof(y))

4.679853735636909286562544242024335307975e-45
ArbReal{137}


## Plotting

# **Making sure Bessels and `solve()` work in arbitrary precision**

# **Using parametric types to easily switch between Float64 and ArbFloat**

In [48]:
setprecision(T, 52); setextrabits(0);

In [ ]:
# test besselk overflow limits
T = Float64
nu = T(1.0)
for x_val in [10.0, 50.0, 100.0, 500.0, 1000.0, 5000.0, 10000.0]
    x = T(x_val)
    try
        y = besseli(nu, x)
        println("x = $x_val: besselk = $y")
    catch e
        println("x = $x_val: ERROR - $e")
        break
    end
end


x = 10.0: besselk = 2670.988303701255
x = 50.0: besselk = 2.903078590103557e20
x = 100.0: besselk = 1.0683693903381627e42
x = 500.0: besselk = 2.5023034121761e215
x = 1000.0: ERROR - SpecialFunctions.AmosException(2)


ErrorException: type AmosException has no field msg

In [50]:
# test besselk overflow limits
T = ArbReal
nu = T("1.0")
for x_val in [10.0, 50.0, 100.0, 500.0, 1000.0, 5000.0, 10000.0]
    x = T(string(x_val))
    try
        y = besseli(nu, x)
        println("x = $x_val: besselk = $y")
    catch e
        println("x = $x_val: ERROR - $e")
        break
    end
end


x = 10.0: besselk = 2670.9883037013
x = 50.0: besselk = 2.9030785901036e+20
x = 100.0: besselk = 1.06836939033816e+42
x = 500.0: besselk = 2.5023034121761e+215
x = 1000.0: besselk = 2.4844429420059e+432
x = 5000.0: besselk = 1.67417944138048e+2169
x = 10000.0: besselk = 3.5132803889537e+4340
